::::
:::{thebe-button}
:::
::::

# Fit tracks

In [ ]:
from __future__ import annotations

from boilercore.fits import Fit, fit_from_params
from boilercv_pipeline.models.path import get_datetime
from boilercv_pipeline.stages import get_thermal_data
from boilercv_pipeline.stages.find_tracks import FindTracks as Params
from dev.docs.nbs import init
from devtools import pprint
from more_itertools import one
from numpy import inf
from pandas import concat, read_hdf

from boilercv.dimensionless_params import jakob, prandtl

In [ ]:
params = Params(context=init(), only_sample=True)
params.set_display_options()
C = params.cols

thermal = read_hdf(params.deps.thermal)
TC = get_thermal_data.Cols()

all_tracks = list(one(params.dfs).parent.iterdir())
times = [get_datetime(p.stem) for p in all_tracks]

# Physical parameters
LATENT_HEAT_OF_VAPORIZATION = 2.23e6  # J/kg
LIQUID_DENSITY = 960  # kg/m^3
LIQUID_DYNAMIC_VISCOSITY = 2.88e-4  # Pa-s
LIQUID_ISOBARIC_SPECIFIC_HEAT = 4213  # J/kg-K
LIQUID_THERMAL_CONDUCTIVITY = 0.676  # W/m-K
VAPOR_DENSITY = 0.804  # kg/m^3

# Plotting
MAX_FOURIER = 0.005
"""Maximum Fourier number to plot."""
MAX_BETA = 1.00
"""Maximum dimensionless bubble diameter to plot."""
MAX_NUSSELT = 1000
"""Maximum Nusselt number to plot."""

C_2 = 0.61
C_3 = 0.33

pprint(params)

In [ ]:
tracks = (
    concat(
        read_hdf(p, key="dst").assign(**{TC.time(): time})
        for p, time in zip(all_tracks, times, strict=True)
    )
    .set_index(TC.time())
    .assign(**{
        TC.subcool(): thermal.set_index(TC.time()).loc[times, TC.subcool()],  # pyright: ignore[reportArgumentType, reportCallIssue]
        "jakob": lambda df: jakob(
            liquid_density=LIQUID_DENSITY,
            vapor_density=VAPOR_DENSITY,
            liquid_isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
            subcooling=df[TC.subcool()],
            latent_heat_of_vaporization=LATENT_HEAT_OF_VAPORIZATION,
        ),
    })
    .reset_index()
)
Pr = prandtl(
    dynamic_viscosity=LIQUID_DYNAMIC_VISCOSITY,
    isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
    thermal_conductivity=LIQUID_THERMAL_CONDUCTIVITY,
)
Ja = tracks["jakob"].median()
Re_b0 = tracks[C.bub_reynolds0()].median()


def Nu(Re_b, C_1, C_4):  # noqa: N803, D103, N802
    return C_1 * Re_b**C_2 * Pr**C_3 * Ja**C_4


def Nu2(x, C_1, C_2, C_3, C_4):  # noqa: N803, D103, N802
    Re_b, Ja = x.T  # noqa: N806
    return C_1 * Re_b**C_2 * Pr**C_3 * Ja**C_4

In [ ]:
fits, errors = fit_from_params(
    model=Nu,
    params=Fit(
        independent_params=(["Re_b"]),
        free_params=(["C_1", "C_4"]),
        values={"C_1": 1.0, "C_4": 1.0},
        bounds={"C_1": (-inf, inf), "C_4": (-inf, inf)},
    ),
    x=tracks[C.bub_reynolds()].values,
    y=tracks[C.bub_nusselt()].values,
)
display(
    dict(
        sorted(
            {
                "C_2": C_2,
                "C_3": C_3,
                **fits,
                "C_5": 2 * fits["C_1"] * (2 - C_2),
                "C_6": 1 + fits["C_4"],
                "C_7": 1 / (2 - C_2),
            }.items()
        )
    ),
    errors,
)

In [ ]:
fits, errors = fit_from_params(
    model=Nu,
    params=Fit(
        independent_params=(["Re_b"]),
        free_params=(["C_1", "C_4"]),
        values={"C_1": 1.0, "C_4": 1.0},
        bounds={"C_1": (0, inf), "C_4": (0, inf)},
    ),
    x=tracks[C.bub_reynolds()].values,
    y=tracks[C.bub_nusselt()].values,
)
display(
    dict(
        sorted(
            {
                "C_2": C_2,
                "C_3": C_3,
                **fits,
                "C_5": 2 * fits["C_1"] * (2 - C_2),
                "C_6": 1 + fits["C_4"],
                "C_7": 1 / (2 - C_2),
            }.items()
        )
    ),
    errors,
)

In [ ]:
fits, errors = fit_from_params(
    model=Nu2,
    params=Fit(
        independent_params=(["x"]),
        free_params=(["C_1", "C_2", "C_3", "C_4"]),
        values={"C_1": 1.0, "C_2": 1.0, "C_3": 1.0, "C_4": 1.0},
        bounds={
            "C_1": (-inf, inf),
            "C_2": (-inf, inf),
            "C_3": (-inf, inf),
            "C_4": (-inf, inf),
        },
    ),
    x=tracks[[C.bub_reynolds(), "jakob"]].values,
    y=tracks[C.bub_nusselt()].values,
)
display(
    dict(
        sorted(
            {
                **fits,
                "C_5": 2 * fits["C_1"] * (2 - C_2),
                "C_6": 1 + fits["C_4"],
                "C_7": 1 / (2 - C_2),
            }.items()
        )
    ),
    errors,
)

In [ ]:
fits, errors = fit_from_params(
    model=Nu2,
    params=Fit(
        independent_params=(["x"]),
        free_params=(["C_1", "C_2", "C_3", "C_4"]),
        values={"C_1": 1.0, "C_2": 1.0, "C_3": 1.0, "C_4": 1.0},
        bounds={
            "C_1": (0.0, inf),
            "C_2": (0.0, inf),
            "C_3": (0.0, inf),
            "C_4": (0.0, inf),
        },
    ),
    x=tracks[[C.bub_reynolds(), "jakob"]].values,
    y=tracks[C.bub_nusselt()].values,
)
display(
    dict(
        sorted(
            {
                **fits,
                "C_5": 2 * fits["C_1"] * (2 - C_2),
                "C_6": 1 + fits["C_4"],
                "C_7": 1 / (2 - C_2),
            }.items()
        )
    ),
    errors,
)